# Phase 4: Generative AI

<div style="font-family: 'Times New Roman', Times, serif; font-size: 18px;">
In this phase, we integrate Generative AI into our Job Recommendadtion System. This AI integration will improve the Job Recommendadtion System by providing comparative and actionable templates. The comparative template will provide comparisions between recommended job titles while the actionable template will provide step-by-step guide how to pursue these job titled. We will use the LLAMA Generative AI model to implement these templates and improve our job recommendadtion system.
</div>

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from groq import Groq
import warnings
warnings.filterwarnings("ignore")

## Load datasets and models

In [2]:
# read datasets 
# read transformed dataset to get sample input data from it.
transformed_df = pd.read_csv('Dataset/transformed_job_dataset.csv')

# read groupd_data to get similar job titles to current input data
grouped_data = pd.read_csv('Dataset/grouped_data.csv')

# read k-means model to find similar job titles
KmeansModel = pickle.load(open('KmeansModel.sav', 'rb')) 

## Get example input data

In [3]:
# features
features = transformed_df.drop(['job_title'], axis = 1)

# sample input data
input_data = features.iloc[550,:]

## Find Job Title Cluster

In [4]:
# find cluster of the current input data
pred = KmeansModel.predict(np.array(input_data).reshape(1, -1))
predicted_cluster = pred[0] 

# Make Recommendations

In [5]:
# Retrieve job titles in the same cluster
job_titles = grouped_data[grouped_data['group_labels'] == predicted_cluster]
top_job_titles = job_titles['job_title'].value_counts().head(3).index.tolist()

In [6]:
print('We recommend the following Job Titles: ')
for job_title in top_job_titles:
    print(job_title)

We recommend the following Job Titles: 
sales manager
salesperson
seasonal sales associate


# Generative AI Integration 

## Load libraries

## API Key

In [7]:
os.environ["GROQ_API_KEY"] = "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"

# Comparative Template

In [8]:
def recommendation_comparative(cluster, job_titles):  
    top_job_titles = job_titles['job_title'].value_counts().head(3).index.tolist()
        
    gen_ai_prompt = f"""
    You are an AI Job Recommendation assistant. Based on the user's skills and cluster {cluster}, here is a comparison of their profile with the recommended job titles:

    - Recommended Job Titles: {', '.join(top_job_titles)}

    Comparison:
    1. **Skill Alignment**: These roles align with your current skills in [specific skills].
    2. **Growth Opportunities**: These titles offer opportunities to develop [specific new skills].
    3. **Industry Trends**: These roles are in high demand in [specific industries].

    This comparison will help you make an informed decision about your career path.
    """
    
    # Use LLAMA Gen AI 
    gen_ai_model = Groq(api_key=os.environ.get("GROQ_API_KEY"), )

    output = gen_ai_model.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "system", "content": "You are a job recommendation assistant."},
                  {"role": "user", "content": gen_ai_prompt}],
        temperature=1.25,
        max_tokens=1500
    )
           
    return output.choices[0].message.content

# Actionable Template

In [9]:
def recommendation_actionable(cluster, job_titles):  
    top_job_titles = job_titles['job_title'].value_counts().head(3).index.tolist()
        
    gen_ai_prompt = f"""
    You are an AI Job Recommendation assistant. Based on your skills and cluster {cluster}, here are the recommended job titles and actionable steps to pursue them:

    - Recommended Job Titles: {', '.join(top_job_titles)}

    Actionable Steps:
    1. **Upskill**: Consider learning [specific skills] to better align with these roles.
    2. **Network**: Connect with professionals in [specific industries] on platforms like LinkedIn.
    3. **Tailor Your Resume**: Highlight your experience in [specific areas] to stand out for these roles.
    4. **Apply Strategically**: Focus on companies like [specific companies] that value your skill set.

    Follow these steps to increase your chances of landing your desired role.
    """
    
    # Create the Groq client
    gen_ai_model = Groq(api_key=os.environ.get("GROQ_API_KEY"), )

    output = gen_ai_model.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "system", "content": "You are a job recommendation assistant."},
                  {"role": "user", "content": gen_ai_prompt}],
        temperature=1,
        max_tokens=1024
    )
           
    return output.choices[0].message.content

### Comparative Recommendation

In [10]:
recommendation1 = recommendation_comparative(predicted_cluster, job_titles)

print("--- Comparative Recommendation ---")
print(recommendation1)


--- Comparative Recommendation ---
To assist you in making an informed decision, I'll help analyze your skills in relation to job roles in cluster 1 (sales-related positions). 

Based on the provided comparison categories, here's an assessment of the recommended job titles against your profile (we will work on creating a simple example):

Let's assume you have the following skills:
1. Verbal/Written Communication Skills
2. Negotiation/ Persuasion 
3. Product Knowledge
4. Analytical Thinking
5. Customer Service

Here is how your skills align with the recommended job titles:

1. **Sales Manager**:
   - **Skill Alignment**: Your verbal communication, negotiation and persuasion, analytical thinking, and product knowledge skills make you a strong candidate for this role. As a sales manager, you will need to motivate your team and analyze sales data, which align with your skills.
   - **Growth Opportunities**: To succeed as a sales manager, you'll likely need to develop strong leadership ski

### Actionable Recommendation

In [11]:
recommendation2 = recommendation_actionable(predicted_cluster, job_titles)
print("\n--- Actionable Recommendation ---")
print(recommendation2)



--- Actionable Recommendation ---
Based on your skills and cluster 1, I have tailored a personalized job recommendation plan for you. Here's a customized approach to help you achieve your goal of becoming a sales professional:

**Recommended Job Titles:**

1. Sales Manager
2. Salesperson
3. Seasonal Sales Associate

**Actionable Steps:**

1. **Upskill**: Consider learning the following specific skills to better align with these roles:
	* Negotiation and persuasion techniques
	* Client communication and relationship-building
	* Sales forecasting and analytics
	* Product knowledge and market research
	* CRM software and sales automation tools
2. **Network**: Connect with professionals in the following industries on platforms like LinkedIn:
	* Retail and e-commerce
	* Technology and software
	* Financial services and banking
3. **Tailor Your Resume**: Highlight your experience in the following areas to stand out for these roles:
	* Sales experience (if you have any)
	* Customer service a

# Explanation

# Difference between the two templates

<div style="font-family: 'Times New Roman', Times, serif; font-size: 18px;">
The Comparative Template provides a detailed comparison between the user's current skills and the recommended job titles, highlighting skill alignment, growth opportunities, and industry trends. It explains how the user's existing skills match the requirements of the recommended roles, identifies areas for skill development, and discusses the industries where these roles are in demand. This template is ideal for users who are exploring career options and need to understand why the recommended roles are a good fit for their profile.
<br><br>
The Actionable Template offers a step-by-step guide for users to pursue the recommended job titles, focusing on upskilling, networking, tailoring their resume, and applying strategically. It provides specific advice on what skills to learn, how to connect with professionals, how to highlight relevant experience, and which companies or platforms to target. This template is best for users who are ready to take action and need practical, concrete steps to achieve their career goals.
<br><br>
The best template depends on the user's needs: the Comparative Template is ideal for early-stage career exploration, as it helps users understand their fit with recommended roles, while the Actionable Template is better for late-stage job seekers who need a clear roadmap to pursue those roles. However, combining both templates can provide the most value, offering users both the why (understanding their fit) and the how (taking actionable steps) to achieve their career goals.

</div>

# Interactive Job Recommendation System 

<div style="font-family: 'Times New Roman', Times, serif; font-size: 18px;"> In this section, we will enhance the system’s interactivity by allowing users to input their skills and experience level directly in natural text format. Based on this input, the system will process the data and generate personalized job recommendations that align with the user’s skills and experience.
<br><br>
To add more value, the system will provide a detailed comparative analysis of the recommended jobs. This analysis will include key job requirements, salary ranges for entry, mid, and senior levels, growth potential, recommended certifications, and market demand—tailored to the user’s skill set and experience level.
<br><br>
Additionally, the system will offer a career action plan, providing specific steps to help users close any skill gaps, pursue relevant certifications, and optimize their job search. It will also include practical advice on improving the user’s resume, LinkedIn profile, and interview preparation.
<br><br>
To enhance the user experience, we will integrate the grouped dataset alongside the transformed dataset, allowing users to input their skills in a natural text format. This will ensure that the system can match user input to the available skills in a more intuitive and user-friendly way, making the process smoother and more accessible.</div>

In [10]:
import pandas as pd
import numpy as np
import pickle
import os
from groq import Groq
import warnings
warnings.filterwarnings("ignore")

# Load datasets and models
transformed_df = pd.read_csv('Dataset/transformed_job_dataset.csv')
grouped_data = pd.read_csv('Dataset/grouped_data.csv')
KmeansModel = pickle.load(open('KmeansModel.sav', 'rb'))

# Set up Groq API
os.environ["GROQ_API_KEY"] = "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"

def get_user_input():
    """Get user skills in natural language and convert to model format"""
    print("\n" + "="*50)
    print("AI JOB RECOMMENDATION SYSTEM".center(50))
    print("="*50)
    
    # Get all skills from the dataset
    all_skills = set()
    for skills in grouped_data['job_title'].str.cat(grouped_data['job_skill_set']).str.lower().str.split(','):
        all_skills.update(s.strip() for s in skills if isinstance(s, str))
    
    print("\nPlease enter your skills (comma separated):")
    print(f"Example: python, data analysis, machine learning\n")
    print(f"Common skills in our system: {', '.join(list(all_skills)[:10])}...\n")
    
    # Get user input
    user_skills_input = input("Your skills: ").lower()
    category = int(input("Your experience level (1-5): "))
    
    # Convert to numeric format
    input_data = {
        'category': category,
        'desc_0': 1 if 'python' in user_skills_input else 0,
        'desc_1': 1 if 'sql' in user_skills_input else 0,
        'desc_2': 1 if 'machine learning' in user_skills_input else 0,
        'desc_3': 1 if 'leadership' in user_skills_input else 0,
        'desc_4': 1 if 'communication' in user_skills_input else 0
    }
    
    # Ensure all required columns are present
    for col in transformed_df.columns:
        if col != 'job_title' and col not in input_data:
            input_data[col] = 0
    
    return pd.Series(input_data), user_skills_input

def find_job_cluster(input_data):
    """Find the cluster for the user's input data"""
    pred = KmeansModel.predict(np.array(input_data).reshape(1, -1))
    return pred[0]

def get_recommendations(predicted_cluster):
    """Get job recommendations based on cluster"""
    job_titles = grouped_data[grouped_data['group_labels'] == predicted_cluster]
    return job_titles['job_title'].value_counts().head(3).index.tolist()

def generate_comparative_analysis(job_titles, user_skills, cluster):
    """Generate balanced comparative analysis"""
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "Provide clear, concise job comparisons with key details."},
            {"role": "user", "content": f"""
            User Profile:
            - Skills: {user_skills}
            - Experience Level: {cluster}
            
            Recommended Jobs: {', '.join(job_titles)}
            
            For each job, provide:
            1. Key Requirements (2-3 most important)
            2. Salary Range (entry to senior)
            3. Growth Potential (next 2 career steps)
            4. Best Certification (1 most valuable)
            5. Current Market Demand (High/Medium/Low)
            """}
        ],
        temperature=0.7,
        max_tokens=600
    )
    return response.choices[0].message.content

def generate_actionable_steps(job_titles, user_skills, cluster):
    """Generate practical action plan"""
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "Provide actionable career advice with specific steps."},
            {"role": "user", "content": f"""
            User Profile:
            - Current Skills: {user_skills}
            - Target Jobs: {', '.join(job_titles)}
            
            For each job, suggest:
            1. Skill Development:
               - 1 critical skill gap to address
               - Recommended learning resource
            2. Career Moves:
               - Best certification to pursue
               - Valuable professional network to join
            3. Job Search:
               - Optimal job platforms
               - Resume improvement tip
            """}
        ],
        temperature=0.7,
        max_tokens=600
    )
    return response.choices[0].message.content

def main():
    # Get user input
    input_data, user_skills = get_user_input()
    
    # Generate recommendations
    cluster = find_job_cluster(input_data)
    recommendations = get_recommendations(cluster)
    
    print("\n" + "="*50)
    print("TOP RECOMMENDATIONS".center(50))
    for i, job in enumerate(recommendations, 1):
        print(f"{i}. {job}")
    
    # Generate AI insights
    print("\n" + "="*50)
    print("JOB COMPARISON".center(50))
    print(generate_comparative_analysis(recommendations, user_skills, cluster))
    
    print("\n" + "="*50)
    print("CAREER ACTION PLAN".center(50))
    print(generate_actionable_steps(recommendations, user_skills, cluster))

if __name__ == "__main__":
    main()


           AI JOB RECOMMENDATION SYSTEM           

Please enter your skills (comma separated):
Example: python, data analysis, machine learning

Common skills in our system: can-do attitude, cost tracking, user-centric approach, training, new business development representativesales, multilingual communication, finance specialist - channel  customer marketingsap, marketing strategies, self-directed, follow-up...



Your skills:  leadership , urgency , data analysis , english
Your experience level (1-5):  5



               TOP RECOMMENDATIONS                
1. business development manager
2. business development representative
3. finance manager

                  JOB COMPARISON                  
Here are the job comparisons with key details:

**Business Development Manager**

1. Key Requirements:
	* Strong leadership and management skills
	* Ability to analyze data to drive business decisions
	* Excellent communication and presentation skills
2. Salary Range:
	* Entry-level: $60,000 - $80,000 per year
	* Mid-level: $80,000 - $110,000 per year
	* Senior-level: $110,000 - $140,000 per year
3. Growth Potential:
	* Next 2 career steps: Senior Business Development Manager, General Manager
4. Best Certification:
	* Certified Business Development Professional (CBDP) by the Association of Professional Consultants (APC)
5. Current Market Demand: High

**Business Development Representative**

1. Key Requirements:
	* Strong communication and interpersonal skills
	* Ability to analyze data to ident